# Backtesting with Backtrader 

In [18]:
import backtrader as bt
import yfinance as yf
import pandas_ta as ta
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

class GenericStrategy(bt.Strategy):
    """
    A generic, configurable strategy that can test various trading approaches
    """
    params = (
        # Strategy type
        ('strategy_type', 'ma_crossover'),  # 'ma_crossover', 'rsi_oversold', 'bollinger', 'momentum'
        
        # Moving Average parameters
        ('ma_short', 20),
        ('ma_long', 50),
        
        # RSI parameters
        ('rsi_period', 14),
        ('rsi_oversold', 30),
        ('rsi_overbought', 70),
        
        # Bollinger Bands parameters
        ('bb_period', 20),
        ('bb_std', 2),
        
        # Momentum parameters
        ('momentum_period', 10),
        ('momentum_threshold', 0.02),  # 2% threshold
        
        # Risk management
        ('stop_loss', 0.05),     # 5% stop loss
        ('take_profit', 0.10),   # 10% take profit
        ('position_size', 0.95), # 95% of available cash
        
        # General
        ('printlog', True),
    )

    def log(self, txt, dt=None):
        if self.params.printlog:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}: {txt}')

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        # Initialize indicators based on strategy type
        if self.params.strategy_type == 'ma_crossover':
            self.sma_short = bt.indicators.SimpleMovingAverage(
                self.datas[0], period=self.params.ma_short)
            self.sma_long = bt.indicators.SimpleMovingAverage(
                self.datas[0], period=self.params.ma_long)
            self.crossover = bt.indicators.CrossOver(self.sma_short, self.sma_long)
            
        elif self.params.strategy_type == 'rsi_oversold':
            self.rsi = bt.indicators.RelativeStrengthIndex(period=self.params.rsi_period)
            
        elif self.params.strategy_type == 'bollinger':
            self.bollinger = bt.indicators.BollingerBands(
                period=self.params.bb_period, devfactor=self.params.bb_std)
            
        elif self.params.strategy_type == 'momentum':
            self.momentum = bt.indicators.Momentum(period=self.params.momentum_period)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price:.2f}, '
                        f'Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}')
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f'SELL EXECUTED, Price: {order.executed.price:.2f}, '
                        f'Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}')
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log(f'OPERATION PROFIT, GROSS: {trade.pnl:.2f}, NET: {trade.pnlcomm:.2f}')

    def next(self):
        if self.order:
            return

        # Strategy logic based on type
        if self.params.strategy_type == 'ma_crossover':
            self._ma_crossover_logic()
        elif self.params.strategy_type == 'rsi_oversold':
            self._rsi_oversold_logic()
        elif self.params.strategy_type == 'bollinger':
            self._bollinger_logic()
        elif self.params.strategy_type == 'momentum':
            self._momentum_logic()

    def _ma_crossover_logic(self):
        """Moving Average Crossover Strategy"""
        if not self.position:
            if self.crossover > 0:  # Golden cross
                size = int(self.broker.getcash() * self.params.position_size / self.dataclose[0])
                self.order = self.buy(size=size)
                self.log(f'BUY CREATE, {self.dataclose[0]:.2f}')
        else:
            if self.crossover < 0:  # Death cross
                self.order = self.sell()
                self.log(f'SELL CREATE, {self.dataclose[0]:.2f}')

    def _rsi_oversold_logic(self):
        """RSI Oversold/Overbought Strategy"""
        if not self.position:
            if self.rsi < self.params.rsi_oversold:
                size = int(self.broker.getcash() * self.params.position_size / self.dataclose[0])
                self.order = self.buy(size=size)
                self.log(f'BUY CREATE (RSI {self.rsi[0]:.1f}), {self.dataclose[0]:.2f}')
        else:
            if self.rsi > self.params.rsi_overbought:
                self.order = self.sell()
                self.log(f'SELL CREATE (RSI {self.rsi[0]:.1f}), {self.dataclose[0]:.2f}')

    def _bollinger_logic(self):
        """Bollinger Bands Mean Reversion Strategy"""
        if not self.position:
            # Buy when price touches lower band
            if self.dataclose[0] <= self.bollinger.lines.bot[0]:
                size = int(self.broker.getcash() * self.params.position_size / self.dataclose[0])
                self.order = self.buy(size=size)
                self.log(f'BUY CREATE (BB Lower), {self.dataclose[0]:.2f}')
        else:
            # Sell when price touches upper band or crosses middle
            if (self.dataclose[0] >= self.bollinger.lines.top[0] or 
                self.dataclose[0] >= self.bollinger.lines.mid[0]):
                self.order = self.sell()
                self.log(f'SELL CREATE (BB Upper/Mid), {self.dataclose[0]:.2f}')

    def _momentum_logic(self):
        """Momentum Strategy"""
        if not self.position:
            # Buy on positive momentum
            if self.momentum[0] > self.params.momentum_threshold:
                size = int(self.broker.getcash() * self.params.position_size / self.dataclose[0])
                self.order = self.buy(size=size)
                self.log(f'BUY CREATE (Momentum {self.momentum[0]:.3f}), {self.dataclose[0]:.2f}')
        else:
            # Sell on negative momentum
            if self.momentum[0] < -self.params.momentum_threshold:
                self.order = self.sell()
                self.log(f'SELL CREATE (Momentum {self.momentum[0]:.3f}), {self.dataclose[0]:.2f}')

def run_strategy_test(symbol='KCHOL.IS', start_date='2024-01-01', end_date='2025-01-01', 
                      strategy_type='ma_crossover', **kwargs):
    """
    Run a backtest with the specified strategy
    
    Available strategy types:
    - 'ma_crossover': Moving average crossover
    - 'rsi_oversold': RSI oversold/overbought
    - 'bollinger': Bollinger bands mean reversion
    - 'momentum': Price momentum strategy
    """
    
    # Download data
    data = yf.download(symbol, start=start_date, end=end_date, auto_adjust=True)
    
    # Handle MultiIndex columns
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = ['_'.join(col).strip() if col[1] else col[0] for col in data.columns.values]
        data = data.rename(columns={
            f'Open_{symbol}': 'Open',
            f'High_{symbol}': 'High', 
            f'Low_{symbol}': 'Low',
            f'Close_{symbol}': 'Close',
            f'Volume_{symbol}': 'Volume'
        })
    
    # Create cerebro engine
    cerebro = bt.Cerebro()
    
    # Add strategy with parameters
    strategy_params = {'strategy_type': strategy_type}
    strategy_params.update(kwargs)
    cerebro.addstrategy(GenericStrategy, **strategy_params)
    
    # Create data feed
    data_feed = bt.feeds.PandasData(
        dataname=data,
        fromdate=datetime.strptime(start_date, '%Y-%m-%d'),
        todate=datetime.strptime(end_date, '%Y-%m-%d')
    )
    cerebro.adddata(data_feed)
    
    # Set broker parameters
    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.001)  # 0.1% commission
    
    # Add analyzers
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
    
    print(f'Starting Portfolio Value: ${cerebro.broker.getvalue():.2f}')
    print(f'Strategy: {strategy_type}')
    print('=' * 50)
    
    # Run backtest
    results = cerebro.run()
    
    # Print results
    strat = results[0]
    print('=' * 50)
    print(f'Final Portfolio Value: ${cerebro.broker.getvalue():.2f}')
    print(f'Total Return: {((cerebro.broker.getvalue() / 100000) - 1) * 100:.2f}%')
    
    # Analyzer results
    try:
        sharpe = strat.analyzers.sharpe.get_analysis().get('sharperatio', 0)
        if sharpe:
            print(f'Sharpe Ratio: {sharpe:.3f}')
    except:
        print('Sharpe Ratio: N/A')
    
    try:
        drawdown = strat.analyzers.drawdown.get_analysis()
        print(f'Max Drawdown: {drawdown["max"]["drawdown"]:.2f}%')
    except:
        print('Max Drawdown: N/A')
    
    try:
        trades = strat.analyzers.trades.get_analysis()
        print(f'Total Trades: {trades.get("total", {}).get("total", 0)}')
        print(f'Winning Trades: {trades.get("won", {}).get("total", 0)}')
        print(f'Losing Trades: {trades.get("lost", {}).get("total", 0)}')
    except:
        print('Trade Analysis: N/A')
    
    return cerebro, results

# Example usage and testing different strategies
if __name__ == '__main__':
    strategies_to_test = [
        ('ma_crossover', {'ma_short': 10, 'ma_long': 30}),
        ('rsi_oversold', {'rsi_period': 14, 'rsi_oversold': 25, 'rsi_overbought': 75}),
        ('bollinger', {'bb_period': 20, 'bb_std': 2}),
        ('momentum', {'momentum_period': 10, 'momentum_threshold': 0.03})
    ]
    
    print("Testing Multiple Strategies on AAPL (2020-2023)")
    print("=" * 60)
    
    results = {}
    for strategy_name, params in strategies_to_test:
        print(f"\n\nTesting {strategy_name.upper()} Strategy:")
        cerebro, result = run_strategy_test(
            symbol='AAPL',
            strategy_type=strategy_name,
            printlog=False,  # Set to True for detailed logs
            **params
        )
        results[strategy_name] = cerebro.broker.getvalue()
    
    print("\n\n" + "=" * 60)
    print("STRATEGY COMPARISON SUMMARY:")
    print("=" * 60)
    for strategy, final_value in results.items():
        total_return = ((final_value / 100000) - 1) * 100
        print(f"{strategy.upper():<15}: ${final_value:>10,.2f} ({total_return:>+6.2f}%)")

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed

Testing Multiple Strategies on AAPL (2020-2023)


Testing MA_CROSSOVER Strategy:
Starting Portfolio Value: $100000.00
Strategy: ma_crossover
Final Portfolio Value: $135531.24
Total Return: 35.53%
Max Drawdown: 11.24%
Total Trades: 1
Winning Trades: 0
Losing Trades: 0


Testing RSI_OVERSOLD Strategy:
Starting Portfolio Value: $100000.00
Strategy: rsi_oversold



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Final Portfolio Value: $144414.21
Total Return: 44.41%
Max Drawdown: 11.21%
Total Trades: 1
Winning Trades: 0
Losing Trades: 0


Testing BOLLINGER Strategy:
Starting Portfolio Value: $100000.00
Strategy: bollinger
Final Portfolio Value: $139149.27
Total Return: 39.15%
Max Drawdown: 9.98%
Total Trades: 1
Winning Trades: 0
Losing Trades: 0


Testing MOMENTUM Strategy:
Starting Portfolio Value: $100000.00
Strategy: momentum
Final Portfolio Value: $126122.73
Total Return: 26.12%
Max Drawdown: 14.18%
Total Trades: 1
Winning Trades: 0
Losing Trades: 0


STRATEGY COMPARISON SUMMARY:
MA_CROSSOVER   : $135,531.24 (+35.53%)
RSI_OVERSOLD   : $144,414.21 (+44.41%)
BOLLINGER      : $139,149.27 (+39.15%)
MOMENTUM       : $126,122.73 (+26.12%)


In [22]:
import backtrader as bt
import yfinance as yf
import pandas_ta as ta
import pandas as pd
from datetime import datetime
import numpy as np

class AdvancedIndicatorStrategy(bt.Strategy):
    """
    Advanced strategy that can use multiple indicators from your list
    """
    params = (
        # Strategy selection
        ('strategy_mode', 'composite'),  # 'composite', 'momentum', 'trend', 'volume', 'volatility'
        
        # Indicator periods
        ('rsi_period', 14),
        ('adx_period', 14),
        ('cci_period', 14),
        ('roc_period', 12),
        ('atr_period', 14),
        ('obv_period', 10),
        ('cmf_period', 20),
        ('macd_fast', 12),
        ('macd_slow', 26),
        ('macd_signal', 9),
        ('bb_period', 20),
        ('bb_std', 2),
        ('ema_period', 200),
        ('vwap_period', 20),
        ('stoch_k', 14),
        ('stoch_d', 3),
        ('stoch_rsi_period', 14),
        
        # Thresholds
        ('rsi_oversold', 30),
        ('rsi_overbought', 70),
        ('adx_strong', 25),
        ('cci_oversold', -100),
        ('cci_overbought', 100),
        ('stoch_oversold', 20),
        ('stoch_overbought', 80),
        
        # Risk management
        ('position_size', 0.95),
        ('stop_loss_pct', 0.05),
        ('take_profit_pct', 0.15),
        
        ('printlog', True),
    )

    def log(self, txt, dt=None):
        if self.params.printlog:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}: {txt}')

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low
        self.datavolume = self.datas[0].volume
        self.order = None
        
        # Initialize available indicators from your data
        # These will be available if you pre-calculate them with pandas_ta
        try:
            self.rsi = self.datas[0].rsi
            self.adx = self.datas[0].adx
            self.cci = self.datas[0].cci
            self.roc = self.datas[0].roc
            self.atr = self.datas[0].atr
            self.obv = self.datas[0].obv
            self.cmf = self.datas[0].cmf
            self.macd = self.datas[0].macd
            self.macd_signal = self.datas[0].macd_signal
            self.macd_hist = self.datas[0].macd_hist
            self.bb_middle = self.datas[0].bb_middle
            self.bb_upper = self.datas[0].bb_upper
            self.bb_lower = self.datas[0].bb_lower
            self.bb_percent = self.datas[0].bb_percent
            self.bb_width = self.datas[0].bb_width
            self.ema_200 = self.datas[0].ema_200
            self.vwap = self.datas[0].vwap
            self.supertrend = self.datas[0].supertrend
            self.st_way = self.datas[0].st_way
            self.stoch_k = self.datas[0].stoch_k
            self.stoch_d = self.datas[0].stoch_d
            self.stoch_rsi_k = self.datas[0].stoch_rsi_k
            self.stoch_rsi_d = self.datas[0].stoch_rsi_d
            
            self.use_external_indicators = True
            self.log("Using pre-calculated indicators")
        except AttributeError as e:
            # Fallback to backtrader built-in indicators
            self.use_external_indicators = False
            self._init_builtin_indicators()
            self.log("Using built-in indicators")

    def _init_builtin_indicators(self):
        """Initialize backtrader built-in indicators as fallback"""
        self.rsi = bt.indicators.RSI(period=self.params.rsi_period)
        self.macd = bt.indicators.MACD(
            period_me1=self.params.macd_fast,
            period_me2=self.params.macd_slow,
            period_signal=self.params.macd_signal
        )
        self.bb = bt.indicators.BollingerBands(
            period=self.params.bb_period,
            devfactor=self.params.bb_std
        )
        self.atr = bt.indicators.ATR(period=self.params.atr_period)
        self.ema_200 = bt.indicators.EMA(period=self.params.ema_period)
        self.stoch = bt.indicators.Stochastic(
            period=self.params.stoch_k,
            period_dfast=self.params.stoch_d
        )
        
        # For built-in indicators, map to expected names
        self.bb_upper = self.bb.lines.top
        self.bb_lower = self.bb.lines.bot
        self.bb_middle = self.bb.lines.mid
        self.macd_signal = self.macd.lines.signal
        self.stoch_k = self.stoch.lines.percK
        self.stoch_d = self.stoch.lines.percD

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price:.2f}')
            else:
                self.log(f'SELL EXECUTED, Price: {order.executed.price:.2f}')
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def next(self):
        if self.order:
            return

        # Skip if we don't have enough data
        if len(self.dataclose) < 50:
            return

        if self.params.strategy_mode == 'composite':
            self._composite_strategy()
        elif self.params.strategy_mode == 'momentum':
            self._momentum_strategy()
        elif self.params.strategy_mode == 'trend':
            self._trend_strategy()
        elif self.params.strategy_mode == 'volume':
            self._volume_strategy()
        elif self.params.strategy_mode == 'volatility':
            self._volatility_strategy()

    def _composite_strategy(self):
        """Multi-indicator composite strategy"""
        if not self.position:
            buy_signals = 0
            total_signals = 0
            
            # RSI oversold
            if hasattr(self, 'rsi') and len(self.rsi) > 0 and not np.isnan(self.rsi[0]):
                if self.rsi[0] < self.params.rsi_oversold:
                    buy_signals += 1
                total_signals += 1
            
            # MACD bullish crossover
            if (hasattr(self, 'macd') and hasattr(self, 'macd_signal') and 
                len(self.macd) > 1 and len(self.macd_signal) > 1):
                if (not np.isnan(self.macd[0]) and not np.isnan(self.macd_signal[0]) and
                    not np.isnan(self.macd[-1]) and not np.isnan(self.macd_signal[-1])):
                    if self.macd[0] > self.macd_signal[0] and self.macd[-1] <= self.macd_signal[-1]:
                        buy_signals += 2  # Weight MACD higher
                    total_signals += 2
            
            # Price above EMA 200 (trend)
            if (hasattr(self, 'ema_200') and len(self.ema_200) > 0 and 
                not np.isnan(self.ema_200[0])):
                if self.dataclose[0] > self.ema_200[0]:
                    buy_signals += 1
                total_signals += 1
            
            # Bollinger Bands - price near lower band
            if (hasattr(self, 'bb_lower') and len(self.bb_lower) > 0 and 
                not np.isnan(self.bb_lower[0])):
                if self.dataclose[0] <= self.bb_lower[0] * 1.02:
                    buy_signals += 1
                total_signals += 1
            
            # Stochastic oversold
            if (hasattr(self, 'stoch_k') and len(self.stoch_k) > 0 and 
                not np.isnan(self.stoch_k[0])):
                if self.stoch_k[0] < self.params.stoch_oversold:
                    buy_signals += 1
                total_signals += 1
            
            # ADX shows strong trend
            if (hasattr(self, 'adx') and len(self.adx) > 0 and 
                not np.isnan(self.adx[0])):
                if self.adx[0] > self.params.adx_strong:
                    buy_signals += 1
                total_signals += 1
            
            # Buy if majority of signals agree and we have enough signals
            if total_signals > 0 and buy_signals >= total_signals * 0.6:  # 60% of signals bullish
                size = int(self.broker.getcash() * self.params.position_size / self.dataclose[0])
                if size > 0:
                    self.order = self.buy(size=size)
                    self.log(f'COMPOSITE BUY: {buy_signals}/{total_signals} signals, Price: {self.dataclose[0]:.2f}')
        
        else:
            sell_signals = 0
            
            # RSI overbought
            if (hasattr(self, 'rsi') and len(self.rsi) > 0 and 
                not np.isnan(self.rsi[0])):
                if self.rsi[0] > self.params.rsi_overbought:
                    sell_signals += 1
                    
            # MACD bearish crossover
            if (hasattr(self, 'macd') and hasattr(self, 'macd_signal') and 
                len(self.macd) > 1 and len(self.macd_signal) > 1):
                if (not np.isnan(self.macd[0]) and not np.isnan(self.macd_signal[0]) and
                    not np.isnan(self.macd[-1]) and not np.isnan(self.macd_signal[-1])):
                    if self.macd[0] < self.macd_signal[0] and self.macd[-1] >= self.macd_signal[-1]:
                        sell_signals += 2
                        
            # Price hits upper Bollinger Band
            if (hasattr(self, 'bb_upper') and len(self.bb_upper) > 0 and 
                not np.isnan(self.bb_upper[0])):
                if self.dataclose[0] >= self.bb_upper[0]:
                    sell_signals += 1
                    
            # Stochastic overbought
            if (hasattr(self, 'stoch_k') and len(self.stoch_k) > 0 and 
                not np.isnan(self.stoch_k[0])):
                if self.stoch_k[0] > self.params.stoch_overbought:
                    sell_signals += 1
            
            if sell_signals >= 2:
                self.order = self.sell()
                self.log(f'COMPOSITE SELL: {sell_signals} signals, Price: {self.dataclose[0]:.2f}')

    def _momentum_strategy(self):
        """Pure momentum strategy using RSI, ROC, Stochastic"""
        if not self.position:
            momentum_score = 0
            
            if (hasattr(self, 'rsi') and len(self.rsi) > 0 and 
                not np.isnan(self.rsi[0]) and self.rsi[0] < 35):
                momentum_score += 1
                
            if (hasattr(self, 'roc') and len(self.roc) > 0 and 
                not np.isnan(self.roc[0]) and self.roc[0] > 0):
                momentum_score += 1
                
            if (hasattr(self, 'stoch_k') and len(self.stoch_k) > 0 and 
                not np.isnan(self.stoch_k[0]) and self.stoch_k[0] < 25):
                momentum_score += 1
                
            if momentum_score >= 2:
                size = int(self.broker.getcash() * self.params.position_size / self.dataclose[0])
                if size > 0:
                    self.order = self.buy(size=size)
                    self.log(f'MOMENTUM BUY: Score {momentum_score}')
        else:
            if (hasattr(self, 'rsi') and len(self.rsi) > 0 and 
                not np.isnan(self.rsi[0]) and self.rsi[0] > 65):
                self.order = self.sell()
                self.log('MOMENTUM SELL: RSI overbought')

    def _trend_strategy(self):
        """Trend following using EMA, ADX, Supertrend"""
        if not self.position:
            trend_bullish = True
            
            # Price above EMA 200
            if (hasattr(self, 'ema_200') and len(self.ema_200) > 0 and 
                not np.isnan(self.ema_200[0])):
                if self.dataclose[0] <= self.ema_200[0]:
                    trend_bullish = False
            else:
                trend_bullish = False
                
            # Strong ADX
            if (hasattr(self, 'adx') and len(self.adx) > 0 and 
                not np.isnan(self.adx[0])):
                if self.adx[0] <= self.params.adx_strong:
                    trend_bullish = False
            else:
                trend_bullish = False
                
            # Supertrend bullish (if available)
            if hasattr(self, 'st_way') and len(self.st_way) > 0:
                if not np.isnan(self.st_way[0]) and self.st_way[0] <= 0:
                    trend_bullish = False
                
            if trend_bullish:
                size = int(self.broker.getcash() * self.params.position_size / self.dataclose[0])
                if size > 0:
                    self.order = self.buy(size=size)
                    self.log('TREND BUY: All trend indicators bullish')
        else:
            # Exit on trend reversal
            if (hasattr(self, 'st_way') and len(self.st_way) > 0 and 
                not np.isnan(self.st_way[0]) and self.st_way[0] <= 0):
                self.order = self.sell()
                self.log('TREND SELL: Supertrend bearish')
            elif (hasattr(self, 'ema_200') and len(self.ema_200) > 0 and 
                  not np.isnan(self.ema_200[0]) and self.dataclose[0] <= self.ema_200[0]):
                self.order = self.sell()
                self.log('TREND SELL: Price below EMA 200')

    def _volume_strategy(self):
        """Volume-based strategy using OBV, CMF"""
        if not self.position:
            if (hasattr(self, 'obv') and hasattr(self, 'cmf') and 
                len(self.obv) > 1 and len(self.cmf) > 0):
                if (not np.isnan(self.obv[0]) and not np.isnan(self.obv[-1]) and
                    not np.isnan(self.cmf[0])):
                    if self.obv[0] > self.obv[-1] and self.cmf[0] > 0.1:
                        size = int(self.broker.getcash() * self.params.position_size / self.dataclose[0])
                        if size > 0:
                            self.order = self.buy(size=size)
                            self.log('VOLUME BUY: OBV rising, CMF positive')
        else:
            if (hasattr(self, 'cmf') and len(self.cmf) > 0 and 
                not np.isnan(self.cmf[0]) and self.cmf[0] < -0.1):
                self.order = self.sell()
                self.log('VOLUME SELL: CMF negative')

    def _volatility_strategy(self):
        """Volatility-based using ATR, Bollinger Bands"""
        if not self.position:
            if (hasattr(self, 'bb_percent') and hasattr(self, 'atr') and 
                len(self.bb_percent) > 0 and len(self.atr) > 5):
                if (not np.isnan(self.bb_percent[0]) and not np.isnan(self.atr[0])):
                    atr_mean = np.mean([self.atr[i] for i in range(-5, 0) if not np.isnan(self.atr[i])])
                    if self.bb_percent[0] < 0.1 and self.atr[0] > atr_mean:
                        size = int(self.broker.getcash() * self.params.position_size / self.dataclose[0])
                        if size > 0:
                            self.order = self.buy(size=size)
                            self.log('VOLATILITY BUY: Low BB%, High ATR')
        else:
            if (hasattr(self, 'bb_percent') and len(self.bb_percent) > 0 and 
                not np.isnan(self.bb_percent[0]) and self.bb_percent[0] > 0.9):
                self.order = self.sell()
                self.log('VOLATILITY SELL: High BB%')

def prepare_advanced_data(symbol, start_date, end_date):
    """Prepare data with many of your indicators using pandas_ta"""
    
    # Download data
    data = yf.download(symbol, start=start_date, end=end_date, auto_adjust=True)
    
    # Handle MultiIndex columns
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = ['_'.join(col).strip() if col[1] else col[0] for col in data.columns.values]
        data = data.rename(columns={
            f'Open_{symbol}': 'Open',
            f'High_{symbol}': 'High', 
            f'Low_{symbol}': 'Low',
            f'Close_{symbol}': 'Close',
            f'Volume_{symbol}': 'Volume'
        })
    
    print(f"Calculating indicators for {symbol}...")
    
    # Your indicators that can be calculated with pandas_ta
    try:
        # Momentum indicators
        data['rsi'] = ta.rsi(data['Close'], length=14)
        data['roc'] = ta.roc(data['Close'], length=12)
        data['cci'] = ta.cci(data['High'], data['Low'], data['Close'], length=14)
        
        # Trend indicators  
        adx_data = ta.adx(data['High'], data['Low'], data['Close'], length=14)
        data['adx'] = adx_data['ADX_14'] if 'ADX_14' in adx_data.columns else adx_data.iloc[:, 0]
        data['ema_200'] = ta.ema(data['Close'], length=200)
        
        # Volume indicators
        data['obv'] = ta.obv(data['Close'], data['Volume'])
        data['cmf'] = ta.cmf(data['High'], data['Low'], data['Close'], data['Volume'], length=20)
        
        # Volatility indicators
        data['atr'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
        
        # MACD
        macd_data = ta.macd(data['Close'], fast=12, slow=26, signal=9)
        data['macd'] = macd_data['MACD_12_26_9']
        data['macd_signal'] = macd_data['MACDs_12_26_9']
        data['macd_hist'] = macd_data['MACDh_12_26_9']
        
        # Bollinger Bands
        bb_data = ta.bbands(data['Close'], length=20, std=2)
        data['bb_lower'] = bb_data['BBL_20_2.0']
        data['bb_middle'] = bb_data['BBM_20_2.0']
        data['bb_upper'] = bb_data['BBU_20_2.0']
        data['bb_percent'] = bb_data['BBP_20_2.0']
        data['bb_width'] = bb_data['BBB_20_2.0']
        
        # VWAP
        data['vwap'] = ta.vwap(data['High'], data['Low'], data['Close'], data['Volume'])
        
        # Supertrend
        try:
            supertrend_data = ta.supertrend(data['High'], data['Low'], data['Close'], length=7, multiplier=3)
            data['supertrend'] = supertrend_data[f'SUPERT_7_3.0']
            data['st_way'] = supertrend_data[f'SUPERTd_7_3.0']
        except:
            print("Supertrend calculation failed, skipping...")
        
        # Stochastic
        stoch_data = ta.stoch(data['High'], data['Low'], data['Close'], k=14, d=3)
        data['stoch_k'] = stoch_data['STOCHk_14_3_3']
        data['stoch_d'] = stoch_data['STOCHd_14_3_3']
        
        # Stochastic RSI
        try:
            stoch_rsi_data = ta.stochrsi(data['Close'], length=14, rsi_length=14, k=3, d=3)
            data['stoch_rsi_k'] = stoch_rsi_data['STOCHRSIk_14_14_3_3']
            data['stoch_rsi_d'] = stoch_rsi_data['STOCHRSId_14_14_3_3']
        except:
            print("StochRSI calculation failed, skipping...")
        
        print(f"Successfully calculated indicators")
        
    except Exception as e:
        print(f"Error calculating some indicators: {e}")
    
    # Drop NaN values
    initial_rows = len(data)
    data = data.dropna()
    print(f"Data shape: {data.shape} (dropped {initial_rows - len(data)} NaN rows)")
    
    return data

# Create custom data feed class
class AdvancedPandasData(bt.feeds.PandasData):
    """
    Extended PandasData class to handle additional indicator columns
    """
    lines = ('rsi', 'adx', 'cci', 'roc', 'atr', 'obv', 'cmf', 'macd', 
            'macd_signal', 'macd_hist', 'bb_middle', 'bb_upper', 'bb_lower',
            'bb_percent', 'bb_width', 'ema_200', 'vwap', 'supertrend', 
            'st_way', 'stoch_k', 'stoch_d', 'stoch_rsi_k', 'stoch_rsi_d')
    
    params = (
        ('datetime', None),
        ('open', 'Open'),
        ('high', 'High'),
        ('low', 'Low'),
        ('close', 'Close'),
        ('volume', 'Volume'),
        ('openinterest', -1),
        ('rsi', 'rsi'),
        ('adx', 'adx'),
        ('cci', 'cci'),
        ('roc', 'roc'),
        ('atr', 'atr'),
        ('obv', 'obv'),
        ('cmf', 'cmf'),
        ('macd', 'macd'),
        ('macd_signal', 'macd_signal'),
        ('macd_hist', 'macd_hist'),
        ('bb_middle', 'bb_middle'),
        ('bb_upper', 'bb_upper'),
        ('bb_lower', 'bb_lower'),
        ('bb_percent', 'bb_percent'),
        ('bb_width', 'bb_width'),
        ('ema_200', 'ema_200'),
        ('vwap', 'vwap'),
        ('supertrend', 'supertrend'),
        ('st_way', 'st_way'),
        ('stoch_k', 'stoch_k'),
        ('stoch_d', 'stoch_d'),
        ('stoch_rsi_k', 'stoch_rsi_k'),
        ('stoch_rsi_d', 'stoch_rsi_d'),
    )

def run_advanced_backtest(symbol='AAPL', strategy_mode='composite', **kwargs):
    """Run backtest with advanced multi-indicator strategy"""
    
    data = prepare_advanced_data(symbol, '2024-01-01', '2025-01-01')
    
    cerebro = bt.Cerebro()
    cerebro.addstrategy(AdvancedIndicatorStrategy, strategy_mode=strategy_mode, **kwargs)
    
    # Only include columns that exist in the data
    available_columns = data.columns.tolist()
    indicator_columns = ['rsi', 'adx', 'cci', 'roc', 'atr', 'obv', 'cmf', 'macd', 
                        'macd_signal', 'macd_hist', 'bb_middle', 'bb_upper', 'bb_lower',
                        'bb_percent', 'bb_width', 'ema_200', 'vwap', 'supertrend', 
                        'st_way', 'stoch_k', 'stoch_d', 'stoch_rsi_k', 'stoch_rsi_d']
    
    # Filter to only include available indicators
    available_indicators = [col for col in indicator_columns if col in available_columns]
    print(f"Available indicators: {available_indicators}")
    
    data_feed = AdvancedPandasData(dataname=data)
    cerebro.adddata(data_feed)
    
    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.001)
    
    # Add analyzers
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
    
    print(f'Starting Portfolio: $100,000')
    print(f'Strategy Mode: {strategy_mode}')
    print('=' * 50)
    
    results = cerebro.run()
    
    final_value = cerebro.broker.getvalue()
    total_return = ((final_value / 100000) - 1) * 100
    
    print('=' * 50)
    print(f'Final Portfolio: ${final_value:,.2f}')
    print(f'Total Return: {total_return:.2f}%')
    
    # Print analyzer results
    strat = results[0]
    try:
        trades = strat.analyzers.trades.get_analysis()
        total_trades = trades.get("total", {}).get("total", 0)
        won_trades = trades.get("won", {}).get("total", 0)
        if total_trades > 0:
            win_rate = (won_trades / total_trades) * 100
            print(f'Total Trades: {total_trades}')
            print(f'Win Rate: {win_rate:.1f}%')
        else:
            print('No trades executed')
    except Exception as e:
        print(f"Error getting trade analysis: {e}")
        
    try:
        sharpe = strat.analyzers.sharpe.get_analysis().get('sharperatio', 'N/A')
        if sharpe != 'N/A':
            print(f'Sharpe Ratio: {sharpe:.3f}')
    except:
        pass
        
    return cerebro, results

if __name__ == '__main__':
    # Test different strategy modes
    modes = ['composite', 'momentum', 'trend', 'volume', 'volatility']
    
    for mode in modes:
        print(f"\n{'='*60}")
        print(f"TESTING {mode.upper()} STRATEGY")
        print('='*60)
        try:
            run_advanced_backtest('KCHOL.IS', strategy_mode=mode, printlog=False)
        except Exception as e:
            print(f"Error running {mode} strategy: {e}")
            continue

[*********************100%***********************]  1 of 1 completed


TESTING COMPOSITE STRATEGY
Calculating indicators for KCHOL.IS...
Successfully calculated indicators
Data shape: (50, 28) (dropped 199 NaN rows)
Available indicators: ['rsi', 'adx', 'cci', 'roc', 'atr', 'obv', 'cmf', 'macd', 'macd_signal', 'macd_hist', 'bb_middle', 'bb_upper', 'bb_lower', 'bb_percent', 'bb_width', 'ema_200', 'vwap', 'supertrend', 'st_way', 'stoch_k', 'stoch_d', 'stoch_rsi_k', 'stoch_rsi_d']
Starting Portfolio: $100,000
Strategy Mode: composite
Final Portfolio: $100,000.00
Total Return: 0.00%
No trades executed

TESTING MOMENTUM STRATEGY



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Calculating indicators for KCHOL.IS...
Successfully calculated indicators
Data shape: (50, 28) (dropped 199 NaN rows)
Available indicators: ['rsi', 'adx', 'cci', 'roc', 'atr', 'obv', 'cmf', 'macd', 'macd_signal', 'macd_hist', 'bb_middle', 'bb_upper', 'bb_lower', 'bb_percent', 'bb_width', 'ema_200', 'vwap', 'supertrend', 'st_way', 'stoch_k', 'stoch_d', 'stoch_rsi_k', 'stoch_rsi_d']
Starting Portfolio: $100,000
Strategy Mode: momentum
Final Portfolio: $100,000.00
Total Return: 0.00%
No trades executed

TESTING TREND STRATEGY
Calculating indicators for KCHOL.IS...
Successfully calculated indicators
Data shape: (50, 28) (dropped 199 NaN rows)
Available indicators: ['rsi', 'adx', 'cci', 'roc', 'atr', 'obv', 'cmf', 'macd', 'macd_signal', 'macd_hist', 'bb_middle', 'bb_upper', 'bb_lower', 'bb_percent', 'bb_width', 'ema_200', 'vwap', 'supertrend', 'st_way', 'stoch_k', 'stoch_d', 'stoch_rsi_k', 'stoch_rsi_d']
Starting Portfolio: $100,000
Strategy Mode: trend


[*********************100%***********************]  1 of 1 completed


Final Portfolio: $100,000.00
Total Return: 0.00%
No trades executed

TESTING VOLUME STRATEGY
Calculating indicators for KCHOL.IS...
Successfully calculated indicators
Data shape: (50, 28) (dropped 199 NaN rows)
Available indicators: ['rsi', 'adx', 'cci', 'roc', 'atr', 'obv', 'cmf', 'macd', 'macd_signal', 'macd_hist', 'bb_middle', 'bb_upper', 'bb_lower', 'bb_percent', 'bb_width', 'ema_200', 'vwap', 'supertrend', 'st_way', 'stoch_k', 'stoch_d', 'stoch_rsi_k', 'stoch_rsi_d']
Starting Portfolio: $100,000
Strategy Mode: volume
Final Portfolio: $100,000.00
Total Return: 0.00%
No trades executed

TESTING VOLATILITY STRATEGY


[*********************100%***********************]  1 of 1 completed


Calculating indicators for KCHOL.IS...
Successfully calculated indicators
Data shape: (50, 28) (dropped 199 NaN rows)
Available indicators: ['rsi', 'adx', 'cci', 'roc', 'atr', 'obv', 'cmf', 'macd', 'macd_signal', 'macd_hist', 'bb_middle', 'bb_upper', 'bb_lower', 'bb_percent', 'bb_width', 'ema_200', 'vwap', 'supertrend', 'st_way', 'stoch_k', 'stoch_d', 'stoch_rsi_k', 'stoch_rsi_d']
Starting Portfolio: $100,000
Strategy Mode: volatility
Final Portfolio: $100,000.00
Total Return: 0.00%
No trades executed
